# Pre_processing

In [ ]:
## Check out working directory
%pwd

In [ ]:
## Import module 

import loader_shoes
import os 
import re
import cv2
import csv
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

## Resize

In [ ]:
## Resize & create file by order:asc
# for modeling
    
path_raw = "E:\\image\\shoes\\##resize_ver4\\d_raw"
path_re = "E:\\image\\shoes\\##resize_ver4\\d_resize"

loader_shoes.resize(path_raw, path_re, height=224, width=224)  #default = 128*128

## Load X : Features

In [ ]:
## Load X : Features

path_re = "E:\\image\\shoes\\##resize_ver4\\d_resize"

resize = loader_shoes.image_load(path_re)

In [ ]:
## Check out first img(resized)

img = resize[0, : ]
plt.figure()
plt.imshow(img)
print(resize.shape)  

In [ ]:
## Make X (feature)

X = resize
X.shape  

## Create Label

### Create Csv file

In [ ]:
## Make csv file in path_re

path_re = "E:\\image\\shoes\\##resize_ver4\\d_re.csv"

# loader_shoes.csv_maker_10(path_re, k1=498, k2=495, k3=496, k4=498, k5=500,
#                           k6=778, k7=533, k8=636, k9=592, k10=675)


loader_shoes.csv_maker_5(path_re, k1=500, k2=500, k3=500, k4=500, k5=500)


### Load y : label

In [ ]:
import loader_shoes

path_re = "E:\\image\\shoes\\##resize_ver4\\d_re.csv"

#y = loader_shoes.label_load(path_re,label_cnt=10)  #label_cnt = len(신발종류)
y = loader_shoes.label_load(path_re,label_cnt=5)  #label_cnt = len(신발종류)
y.shape

## Scailing

In [ ]:
X = X.astype('float')
X = X/255
X.shape  

## Check X, y

In [ ]:
## Confirm X, y
print(X.shape)  
print(y.shape, end='\n\n\n')  

# print("#####Check out : X#####")
# print(X, end='\n\n\n')
# print("#####Check out : y#####")
# print(y)

# Modeling

## VGG16

### Import module 

In [ ]:
## module import
import tensorflow as tf # tensorflow 2.0
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Activation
from tensorflow.keras.layers import Dropout, BatchNormalization, Dense
from tensorflow.keras.optimizers import Adam


from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np


## Transfer learning (전이학습)
from tensorflow.keras.applications.vgg16 import VGG16                        
from tensorflow.keras.applications.vgg19 import VGG19
# from tensorflow.keras.applications.inception_v3 import InceptionV3
# from tensorflow.keras.applications.resnet import ResNet50

In [ ]:
## Check tensorFlow version

print (tf.__version__)

### Validation data (Train:val=0.7:0.3)

In [ ]:
# 훈련/테스트 데이터를 0.7/0.3의 비율로 분리합니다.
x_train, x_val, y_train, y_val = train_test_split(X, y, 
                                                test_size = 0.3, 
                                                random_state = 777)

# Checkout
print(x_train.shape)
print(x_val.shape)  
print(y_train.shape)
print(y_val.shape)  

### Set Vgg16 options

In [ ]:
## VGG16 options

vgg16 = VGG16(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))
vgg16.summary()

In [ ]:
## 가중치 초기값 : imagenet
# layer.trainable=True : 동결 해제 (default)
# layer.trainable=False : 동결 (option)

for layer in vgg16.layers[:15]:
    layer.trainable = False

In [ ]:
vgg16.summary()

### Vgg16 뒷부분 신경망 생성

In [ ]:
# 신경망 객체 생성
model = Sequential()

# stacking vgg16
model.add(vgg16)

# Reshape : Flatten 
model.add(Flatten())

# 완전연결계층1
model.add(Dense(256, kernel_regularizer=tf.keras.regularizers.l2(1.5)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# 완전연결계층2
model.add(Dense(128, kernel_regularizer=tf.keras.regularizers.l2(1.5)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# 출력층(softmax)
#model.add(Dense(10))  # class : 10
model.add(Dense(5))  # class : 5
model.add(Activation('softmax'))

# Check out model 
model.summary()

### Compile

In [ ]:
model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(lr = 0.0001), 
              metrics=['accuracy'])

### fitting

In [ ]:
## Parameters
epochs = 6
batch_size=100
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)  #Early stopping
#https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping
    

## Fitting
hist = model.fit(x_train, y_train, 
                 validation_data=(x_val, y_val), 
                 epochs=epochs, 
                 batch_size=batch_size,
                 callbacks = [callback])

In [ ]:
## Check out accuracy (with independent data)
scores3 = model.evaluate(ind_X, ind_y, verbose=0)

print("Vgg16 ind_dataset Error : %.2f%%" % (100-scores3[1]*100))
print("Vgg16 ind_dataset Error : %.2f" % (scores3[0]))

### Visualization

In [ ]:
# list all data in history
print(hist.history.keys())

In [ ]:
## log : Accuracy
from pandas import DataFrame

Sample = [hist.history['accuracy'],hist.history['val_accuracy']]
df = DataFrame(Sample).transpose()
df.columns = ['train_Acc', 'val_Acc']
df

In [ ]:
## log : Loss 
Sample2 = [hist.history['loss'],hist.history['val_loss']]

df2 = DataFrame(Sample2).transpose()
df2.columns = ['train_Loss', 'val_loss']
df2

In [ ]:
## Visualization : Accuracy
plt.plot(hist.history['accuracy'], color='blue')
plt.plot(hist.history['val_accuracy'], color='red')
plt.title('Vgg16 : Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

## Visualization : Loss
plt.plot(hist.history['loss'], color='blue')
plt.plot(hist.history['val_loss'], color='red')
plt.title('Vgg16 : Loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

### Results

In [ ]:
## Check out accuracy

scores = model.evaluate(x_train, y_train, verbose=0)
scores2 = model.evaluate(x_val, y_val, verbose=0)

print("Vgg16 train Error : %.2f%%" % (100-scores[1]*100))
print("Vgg16 val Error : %.2f%%" % (100-scores2[1]*100))
print(' ')
print("Vgg16 train Loss : %.2f" % (scores[0]))
print("Vgg16 val Loss : %.2f" % (scores2[0]))

### Save model : .h5 file

In [ ]:
## Save model : .h5(Hdf5 type file)

save_path = "e:\\shoes_vgg16_d.h5"
save_model(model, save_path)  

# Testing with 'independent data'

## Resize

In [ ]:
## Resize & create file by order:asc
## for independent testing data

import loader_shoes
ind_path_raw = "E:\\image\\shoes\\##resize_ver4\\ind_d_raw"
ind_path_re = "E:\\image\\shoes\\##resize_ver4\\ind_d_resize"

loader_shoes.resize(ind_path_raw, ind_path_re, height=224, width=224)  #default = 128*128

## Load ind_X

In [ ]:
## Load X : Features

ind_path_re = "E:\\image\\shoes\\##resize_ver4\\ind_d_resize"
ind_resize = loader_shoes.image_load(ind_path_re)

In [ ]:
## Check out first img(resized)
img = ind_resize[0, : ]
plt.figure()
plt.imshow(img)
print(ind_resize.shape)  

In [ ]:
## Make ind_X (feature)
ind_X = ind_resize
ind_X.shape  

## Create Label & Load ind_y

In [ ]:
## Create Label

ind_path_re = "E:\\image\\shoes\\##resize_ver4\\ind_d_re.csv"

# loader_shoes.csv_maker_10(ind_path_re, k1=10, k2=10, k3=10, k4=10, k5=10,
#                          k6=10, k7=10, k8=10, k9=10, k10=10)

loader_shoes.csv_maker_5(ind_path_re, k1=10, k2=10, k3=10, k4=10, k5=10)
                      

In [ ]:
## Load ind_y : Label

ind_path_re = "E:\\image\\shoes\\##resize_ver4\\ind_d_re.csv"

ind_y = loader_shoes.label_load(ind_path_re,label_cnt=5)  #label_cnt = len(신발종류)
ind_y.shape

## Scaling

In [ ]:
ind_X = ind_X.astype('float')
ind_X = ind_X/255
ind_X.shape  

## Check ind_X, ind_y

In [ ]:
## Confirm X, y
print(ind_X.shape)
print(ind_y.shape)  

# print("#####Check out : X#####")
# print(X, end='\n\n\n')
# print("#####Check out : y#####")
# print(y)

## Results

In [ ]:
## Check out Error

scores3 = model.evaluate(ind_X, ind_y, verbose=0)

print("Vgg16 ind_dataset Error : %.2f%%" % (100-scores3[1]*100))